In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import json
from datetime import date, datetime, timedelta
import time
import os
import numpy as np
import requests
from requests.auth import HTTPBasicAuth
import pathlib
#from config import ACCESS_KEY,SECRET_KEY
# import polling
import boto3


# Generating Order
# API Key stored as an env variable
PL_API_KEY = 'fa49aea30c534483bbf74f6ec9bc311a'
PLANET_API_KEY = PL_API_KEY #os.getenv('PL_API_KEY')
orders_v2_url = 'https://api.planet.com/compute/ops/orders/v2'

# set up requests to work with api
auth = HTTPBasicAuth(PLANET_API_KEY, '')
headers = {'content-type': 'application/json'}


item_type = "PSScene4Band"

In [3]:
waypoint_data = {
    'Waypoint' : ['Orangi River', 'Mara River', 'Sand River', 'Grumeti River', 'Lake Masek', 
                  'Lake Magadi', 'Lake Empakaai', 'Lake Magadi 2', 'Mbalageti River', 'Ruwana River', 'Talek River'],
    'latitude' : [-2.302313, -1.562928, -1.595733, -2.249034, -3.024818, -3.202214, -2.915433, -2.656248, -2.603015, -2.044819, -1.416096],
    'longitude' : [34.830777, 34.997068, 35.069241, 34.486842, 35.038474, 35.536431, 35.841355, 34.788239, 34.720511, 34.230374, 35.097661]
}


waypoint_df = pd.DataFrame(waypoint_data)
waypoint_gdf = gpd.GeoDataFrame(waypoint_df, geometry=gpd.points_from_xy(waypoint_df.longitude, waypoint_df.latitude))

# Applying WGS84 to the CRS
waypoint_gdf.crs = {'init' :'epsg:4326'} 

# Converting geodataframe to Meters from Lat/Long
# Allows for square buffer to be applied (450m)
point_gdf_m = waypoint_gdf.to_crs(epsg=3395)

# Applying the buffer, cap_style = 3 --> Square Buffer
buffer = point_gdf_m.buffer(450, cap_style=3)

# Convert buffer back to WGS84 Lat/Long
buffer_wgs84 = buffer.to_crs(epsg=4326)

# Merging GDF and DF to get the Waypoint names
joined_buffer_wgs84 = pd.concat([waypoint_df,buffer_wgs84], axis=1)
joined_buffer_wgs84 = joined_buffer_wgs84.rename(columns = {0:'polygon'}).set_geometry('polygon')
joined_buffer_wgs84_drop = joined_buffer_wgs84.drop(['geometry'], axis=1)
joined_buffer_wgs84_json = joined_buffer_wgs84_drop.to_json()

# transforming to json for inclusion into Planet API
buffer_wgs84_json_parsed = json.loads(joined_buffer_wgs84_json)
buffer_wgs84_json_api = buffer_wgs84_json_parsed['features']#[0]['geometry']['coordinates']


today = datetime.isoformat(datetime.utcnow())+'Z'#(datetime.today())
start_date = datetime.isoformat(datetime.utcnow() - timedelta(7)) + 'Z'

/root/miniconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [11]:
# Getting Image ID's for each waypoint that has the analytic_sr dataset 
# Having to ping the Planet V1 API to return the image id's for our required filter
# Filter variables include: Center Coordinate, Date Range, Cloud Cover, Item Type and Asset Type


def build_order(index):
    geojson_geometry = {
    "type": "Point",
    "coordinates": [
        index['properties']['longitude'], index['properties']['latitude']
        ]
    }

    # get images that overlap with our AOI 
    geometry_filter = {
      "type": "GeometryFilter",
      "field_name": "geometry",
      "config": geojson_geometry
    }

    date_range_filter = {
      "type": "DateRangeFilter",
      "field_name": "acquired",
      "config": {
        "gte": start_date,
        "lte": today
      }
    }

    # only get images which have <10% cloud coverage
    cloud_cover_filter = {
      "type": "RangeFilter",
      "field_name": "cloud_cover",
      "config": {
        "lte": 0.1
      }
    }

    # combine our geo, date, cloud filters
    combined_filter = {
      "type": "AndFilter",
      "config": [geometry_filter, date_range_filter, cloud_cover_filter]
    }
    
    # API request object
    search_request = {
        "interval": "day",
        "item_types": [item_type],
        "asset_types" : "analytic_sr",
        "filter": combined_filter
    }
    
    search_result = \
      requests.post(
        'https://api.planet.com/data/v1/quick-search',
    #     'https://api.planet.com/data/v2',
        auth=HTTPBasicAuth(PLANET_API_KEY, ''),
        json=search_request)

    return search_result



id_list = []

for index in buffer_wgs84_json_api:
    waypoint = index["properties"]["Waypoint"]
    order = build_order(index)
    
    time.sleep(3)
    order = order.json()['features']
    print(len(order))


# appending Image ID to `joined_buffer_wgs84_drop_merge` if the analytic_sr is available
# Will only return image id's that meet this requirement.
    for i in order:
        #print(order)
        if "assets.analytic_sr:download" in order[0]["_permissions"]:
            id_list.append((waypoint,order[0]["id"]))

id_list = list(set(id_list))

4
4
3
3
1
0
0
1
2
1
4


In [12]:
# Merging image id's to the dataframe to maintain continuity

image_ids = pd.DataFrame(np.asarray(id_list))
image_ids.rename(columns = {0:'Waypoint', 1:'Image_ID'}, inplace = True) 
joined_buffer_wgs84_drop_merge = pd.merge(image_ids, joined_buffer_wgs84_drop, on='Waypoint')

In [13]:
# Converting list of tuple polygons to list of lists polygons
# This step is necessary to pull the Geometry from `joined_buffer_wgs84_drop_merge`
# and convert to a list of lists...appending to `joined_buffer_wgs84_drop_merge`.

def coord_lister(geom):
    coords = list(geom.exterior.coords)
    return (coords)

coordinates = joined_buffer_wgs84_drop_merge.polygon.apply(coord_lister)
res = []
for poly in coordinates:
    res_2 = list(map(list, poly)) 
    res.append(res_2)

joined_buffer_wgs84_drop_merge['poly_list'] = res
# joined_buffer_wgs84_drop_merge will be used as the basis for all remaining functions

In [14]:
# Creating the URLs to activate the images...prevents latency during download
id0_url = []
for id0 in joined_buffer_wgs84_drop_merge['Image_ID']:
    id0_url_2 = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0)
    id0_url.append(id0_url_2)

# id0_url


In [15]:
# Returns JSON metadata for assets in this ID. 
# Learn more: planet.com/docs/reference/data-api/items-assets/#asset
result = []
for link in id0_url:
#     print(link)
    result1 = \
      requests.get(
        link,
        auth=HTTPBasicAuth(PLANET_API_KEY, '')
      )
    result.append(result1)

# Parse out useful links

links = []
activation_link = []

# Getting Result Links
for r in result:
    links1 = r.json()[u"analytic_sr"]["_links"]
    links.append(links1)

# Generating a list of activation links    
for l in links:
    activation_link1 = l["activate"]
    activation_link.append(activation_link1)
activation_link

# Request activation of the 'visual' asset:
for a in activation_link:
    activate_result = \
    requests.get(a,auth=HTTPBasicAuth(PLANET_API_KEY, ''))

In [16]:
# Building the order lists starting with the product information

single_product = []
for image_id in joined_buffer_wgs84_drop_merge['Image_ID']:
    single_product_2 = [
        {
          'item_ids': [image_id], 
          'item_type': 'PSScene4Band',
          'product_bundle': 'analytic_sr'
        }
    ]
    single_product.append(single_product_2)
# print(single_product)


In [17]:
# Setting the clipping boundaries

clip = []
for polygon in joined_buffer_wgs84_drop_merge['poly_list']:
    clip_aoi = {
        'type':'Polygon',
        'coordinates': [polygon] 
    }

# define the clip tool
    clip_2 = {
        'clip': {
            'aoi': clip_aoi
        }
    }
    clip.append(clip_2)

In [18]:
# create an order request with the clipping tool
request_clip = []

for p, c in zip(single_product, clip):
    request_clip_2 = {
        'name': 'just clip',
        'products': p, #single_product,
        'tools': [c]
    }
    request_clip.append(request_clip_2)

In [20]:
# Creating the url for clipping 
def place_order(request, auth):
    response = requests.post(orders_v2_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response)
    
    if response.status_code == 429:
        time.sleep(3)
        response = requests.post(orders_v2_url, data=json.dumps(request), auth=auth, headers=headers)
    
        if not response.ok:
            raise Exception(response.content)

    order_id = response.json()['id']
    print(order_id)
    
    order_url = orders_v2_url + '/' + order_id
    
    return order_url


order_url = []
for req_c in request_clip:
    order_url2 = place_order(request_clip[1], auth)
    order_url.append(order_url2)

# order_url

<Response [202]>
710e2691-ea3b-46a1-a98e-8c88d2f2485d
<Response [202]>
2b17d9c9-9ccb-44b7-90ff-732a8c083821
<Response [202]>
1736d836-a48b-4476-8996-fb0bd5456a9d
<Response [202]>
5885147b-c259-4f34-96ea-95d3e4f788fd
<Response [429]>
1a132e22-a92d-4162-86ea-270d89bd74b5
<Response [202]>
5055e6a5-e220-48e0-af6c-dd72c82b066c
<Response [202]>
0efbf449-a4b6-45e9-9880-356cd90e28b6
<Response [429]>
962c435c-e3e3-4169-b79b-4c54dd088b37
<Response [202]>
c1e4a175-5271-44ad-822c-377d7137bc6a


In [22]:
# time.sleep(600)

def poll_for_success(order_url, auth, num_loops=50, sleep_time=10):
    count = 0
    while(count < num_loops):
        r = requests.get(order_url, auth=auth)
        response = r.json()
        state = response['state']
        
        print(f'{count * sleep_time}: {state}')
        success_states = ['success', 'partial']
        
        if state == 'failed':
            raise Exception(response)
        elif state in success_states:
            break
        
        time.sleep(sleep_time)
        count += 1

In [ ]:
run_clip = True

clip_order_url = []
if run_clip:
    for c in order_url:
        time.sleep(20)
        clip_order_url2 = place_order(request_clip[1], auth)
        poll_for_success(clip_order_url2,auth)
        clip_order_url.append(clip_order_url2)

<Response [202]>
2ab69423-8a89-4c0a-9feb-5d0309ba169a
0: queued
10: running
20: running
30: running
40: running
50: running
60: running
70: running
80: running
90: running
100: running
110: running
120: running
130: running
140: running
150: running
160: running
170: running
180: running
190: running
200: running
210: running
220: running
230: running
240: running
250: running
260: running
270: running
280: running
290: running
300: running
310: running
320: running
330: running
340: running
350: running
360: running
370: running
380: running
390: running
400: running
410: running
420: running
430: running
440: running
450: running
460: running
470: running
480: running
490: running
<Response [202]>
e15601bb-5627-4ea9-a2e1-a6adad578470
0: queued
10: queued
20: queued
30: running
40: running
50: running
60: running
70: running
80: running
90: running
100: running
110: running
120: running
130: running
140: running
150: running
160: running
170: running
180: running
190: running
200: run

In [16]:
# Downloading from each clip order url
def download_order(order_url, auth, overwrite=False):
    r = requests.get(order_url, auth=auth)
    print(r)

    response = r.json()
    results = response['_links']['results']
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    results_paths = [pathlib.Path(os.path.join('data', n)) for n in results_names]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name, path in zip(results_urls, results_names, results_paths):
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))
            
    return dict(zip(results_names, results_paths))


# Process sleeping to allow for activation, staging, and processing of imagery


# Downloading the orders
for clip_order in clip_order_url:
    download_order(clip_order, auth)

<Response [200]>
5 items to download
downloading 1ae3d33f-1499-4c77-8066-bbe03385dfec/1/manifest.json to data/1ae3d33f-1499-4c77-8066-bbe03385dfec/1/manifest.json
downloading 1ae3d33f-1499-4c77-8066-bbe03385dfec/1/files/20200312_064310_1053_3B_AnalyticMS_DN_udm_clip.tif to data/1ae3d33f-1499-4c77-8066-bbe03385dfec/1/files/20200312_064310_1053_3B_AnalyticMS_DN_udm_clip.tif
downloading 1ae3d33f-1499-4c77-8066-bbe03385dfec/1/files/20200312_064310_1053_metadata.json to data/1ae3d33f-1499-4c77-8066-bbe03385dfec/1/files/20200312_064310_1053_metadata.json
downloading 1ae3d33f-1499-4c77-8066-bbe03385dfec/1/files/20200312_064310_1053_3B_AnalyticMS_SR_clip.tif to data/1ae3d33f-1499-4c77-8066-bbe03385dfec/1/files/20200312_064310_1053_3B_AnalyticMS_SR_clip.tif
downloading 1ae3d33f-1499-4c77-8066-bbe03385dfec/1/files/20200312_064310_1053_3B_AnalyticMS_metadata_clip.xml to data/1ae3d33f-1499-4c77-8066-bbe03385dfec/1/files/20200312_064310_1053_3B_AnalyticMS_metadata_clip.xml
<Response [200]>
5 items 

In [ ]:
# s3_resource = boto3.resource('s3')
# s3_resource.meta.client.upload_file(
#     Filename='/Users/jasonabaker/W210/Project/Planet_API/Waypoint_Scripts_2/data/e59ba56a-c72a-4f49-b680-273e243f8fd1/1/files/20200305_163004_0f44_3B_AnalyticMS_SR_clip.tif', Bucket='w210-planet-data-api',
#     Key= '20200305_163004_0f44_3B_AnalyticMS_SR_clip.tif')